In [ ]:
import tensorflow.compat.v1 as tf

print(tensorflow.__version__)
#import tensorflow as tf


# mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
#     tf.distribute.experimental.CollectiveCommunication.NCCL)
# print(mirrored_strategy)

# cpu
# WARNING:tensorflow:There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
# INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
# Number of devices: 1

# gpu1
# INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0',)
# INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0',), communication = CollectiveCommunication.NCCL
# <tensorflow.python.distribute.collective_all_reduce_strategy.CollectiveAllReduceStrategy object at 0x7fd0606f4198>


In [ ]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# physical_devices

In [ ]:
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=opts))

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

In [ ]:
# with mirrored_strategy.scope():
with tf.device('/gpu:0'):
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    print(x_train.shape, x_test.shape)
    
    img_rows, img_cols= 28, 28
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
    
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Conv2D(16, kernel_size=(3, 3)),
    #         tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation="relu"),
    #         tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(optimizer='adam',
                  loss=loss_fn,
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=1)